In [ ]:
# Going to use miniwdl and argparse to parse WDL workflows
pip install miniwdl argparse

In [ ]:
# patches asyncio to allow nested use of asyncio.run and loop.run_until_complete
pip install nest_asyncio

In [1]:
import argparse
import json
import WDL
import nest_asyncio
nest_asyncio.apply()

In [2]:
# The top level function to load a WDL file with miniwdl and extract tasks and workflow info. 
def parse_wdl(wdl_file):
    # Load the WDL file using miniwdl
    doc = WDL.load(wdl_file)
    
    tasks = []
    for task in doc.tasks:
        tasks.append(convert_task_to_cwl(task))
    
    workflow = None
    if hasattr(doc, 'workflow'):
        workflow = convert_workflow_to_cwl(doc.workflow)
    
    return tasks, workflow

In [3]:
# Converts tasks in workflow from WDL to CWL by looping through WDL inputs and outputs and adding the to CWL. cwl_tool initiates a dictionary for CWL CommandLineTool.
def convert_task_to_cwl(task):
    cwl_tool = {
        "cwlVersion": "v1.2",
        "class": "CommandLineTool",
        "baseCommand": task.command.parts[0] if task.command else None,
        "inputs": {},
        "outputs": {},
        "requirements": {
            "DockerRequirement": {
                "dockerPull": task.runtime["docker"] if hasattr(task, "runtime") and "docker" in task.runtime else None
            }
        }
    }
    
    # Check if inputs exist before iterating
    if task.inputs:
        for inp in task.inputs:
            cwl_tool["inputs"][inp.name] = {"type": "File" if "File" in str(inp.type) else "string"}  

    # Check if outputs exist before iterating
    if task.outputs:
        for outp in task.outputs:
            cwl_tool["outputs"][outp.name] = {"type": "File"}  # Assuming output is a file
    
    return cwl_tool


In [4]:
#WDL to CWL workflow conversion
def convert_workflow_to_cwl(workflow):
    cwl_workflow = {
        "cwlVersion": "v1.2",
        "class": "Workflow",
        "inputs": {},
        "outputs": {},
        "steps": {}
    }

    # Ensure inputs exist before iterating
    if workflow.inputs:
        for inp in workflow.inputs:
            cwl_workflow["inputs"][inp.name] = {"type": "File" if "File" in str(inp.type) else "string"}

    # Ensure outputs exist before iterating
    if workflow.outputs:
        for outp in workflow.outputs:
            cwl_workflow["outputs"][outp.name] = {"type": "File"}

    # Ensure steps are processed properly
    if workflow.body:
    for element in workflow.body:
        if isinstance(element, WDL.Tree.Call):
            cwl_workflow["steps"][element.name] = {
                "run": f"{element.callee.name}.cwl",
                "in": {},
                "out": list(element.outputs.keys()) if hasattr(element, "outputs") else []
            }
            
    return cwl_workflow


IndentationError: expected an indented block (1867188236.py, line 22)

In [21]:
#Writes CWL dictionary in JSON format
def write_cwl(output_file, content):
    with open(output_file, "w") as f:
        json.dump(content, f, indent=4)



In [22]:
# argparse parses Command-line argumentsand saves workflow if present
def main():
    parser = argparse.ArgumentParser(description="Convert WDL to CWL")
    parser.add_argument("wdl_file", help="Path to the WDL file")
    parser.add_argument("output_file", help="Path to output CWL file")
    args = parser.parse_args()
    
    tasks, workflow = parse_wdl(args.wdl_file)
    
    if workflow:
        write_cwl(args.output_file, workflow)
    else:
        for i, task in enumerate(tasks):
            write_cwl(args.output_file.replace(".cwl", f"_task{i}.cwl"), task)



In [23]:
# Defined path to an example WDL workflow.
wdl_path = "path/to/example"

In [25]:
#Example run
tasks, workflow = parse_wdl(wdl_path)

if workflow:
    print("Converted Workflow:")
    print(json.dumps(workflow, indent=4))
else:
    for i, task in enumerate(tasks):
        print(f"Converted Task {i}:")
        print(json.dumps(task, indent=4))

AttributeError: 'Workflow' object has no attribute 'calls'